#### Calculating Risk parameters on the bank balance
See for a practical guide: https://www.closemountain.com/papers/risktransform1.pdf

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from src.data.zerocurve import Zerocurve
from src.data.interest import Interest
from src.models.bank_model import Bankmodel


In [3]:
zerocurve = Zerocurve()
zerocurve.load_data()
interest = Interest()
interest.load_data()

2023-03-19 10:57:01.783 | INFO     | src.data.dataset:load_data:45 - loading zerocurve data from file.
c:\Users\M\miniconda3\envs\rlalm\lib\site-packages\pandas\core\internals\blocks.py:351: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)
2023-03-19 10:57:02.030 | INFO     | src.data.dataset:load_data:45 - loading interest data from file.


In [4]:
pos_date = zerocurve.df.index[-1]
bankmodel = Bankmodel(pos_date)
bankmodel.generate_mortgage_contracts(n = 250, df_i = interest.df)

2023-03-19 10:57:04.413 | INFO     | src.models.bank_model:generate_mortgage_contracts:111 - Added 250 mortgages to our portfolio.
2023-03-19 10:57:08.853 | INFO     | src.models.bank_model:generate_mortgage_contracts:115 - Added 45442 cashflows to our model.


,category,fixed_period,years,start_date,principal,period,interest,contract
0,3,>10 years,20,2017-06-16,200000,2017-06-01,2.95,0
1,2,5>10 years,10,2018-04-16,200000,2018-04-01,2.33,1
2,3,>10 years,20,2010-02-12,200000,2010-02-01,3.15,2
3,3,>10 years,20,2019-03-07,200000,2019-03-01,2.85,3
4,2,5>10 years,10,2019-03-29,200000,2019-03-01,2.40,4
...,...,...,...,...,...,...,...,...
245,1,1>5 years,5,2021-04-29,200000,2021-04-01,1.55,245
246,2,5>10 years,10,2021-06-24,200000,2021-06-01,1.60,246
247,3,>10 years,20,2014-11-28,200000,2014-11-01,3.84,247
248,2,5>10 years,10,2020-01-07,200000,2020-01-01,1.96,248


In [5]:
result = bankmodel.calculate_npv(zerocurve)
print(f'Net present value {result:,.0f}')

Net present value 37,970,700


##### The standardized framework. 
Supervisory Outlier Test (SOT)

Interest Rate Risk needs to be measured on time buckets
https://www.bis.org/bcbs/publ/d319.pdf
https://www.eba.europa.eu/sites/default/documents/files/document_library/Publications/Draft%20Technical%20Standards/2022/EBA-RTS-2022-10%20RTS%20on%20SOTs/1041756/Final%20draft%20RTS%20on%20SOTs.pdf

1. parallel shock up; 
2. parallel shock down;
3. steepener shock (short rates down and long rates up);
4. flattener shock (short rates up and long rates down); 
5. short rates shock up; and
6. short rates shock down

In [7]:
result = bankmodel.calculate_risk(zerocurve,200,'parallel')
print(f'Parallell shift 200bp up {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,-200,'parallel')
print(f'Parallell shift 200bp down {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,250,'short')
print(f'Short shift 250bp up {result:,.0f}')

result = bankmodel.calculate_risk(zerocurve,100,'long')
print(f'Long shift 100bp up {result:,.0f}')


Parallell shift 200bp up -3,591,894
Parallell shift 200bp down 4,392,786
Short shift 250bp up -194,731
Long shift 100bp up -1,804,303
